In [3]:
!pip install -U datasets -q


[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip


In [1]:
!pip install -U torch

  Using cached torch-2.7.0-cp310-cp310-manylinux_2_28_x86_64.whl.metadata (29 kB)
  Using cached typing_extensions-4.13.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.6.77-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.6.77-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.6.80-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.5.1.17-py3-none-manylinux_2_28_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.6.4.1-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.3.0.4-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.7.77-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.5

In [1]:
!pip install transformers

  Using cached transformers-4.52.1-py3-none-any.whl.metadata (38 kB)
  Using cached regex-2024.11.6-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (40 kB)
  Using cached tokenizers-0.21.1-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.8 kB)
  Using cached safetensors-0.5.3-cp38-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.8 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 80.9 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.7/781.7 kB 130.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 47.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 43.7 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip


In [3]:
!pip install 'accelerate>=0.26'

  Using cached accelerate-1.7.0-py3-none-any.whl.metadata (19 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.1/362.1 kB 4.6 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip


In [ ]:
# !pip install transformers sentencepiece datasets accelerate bitsandbytes -qqq
!pip install -U bitsandbytes

In [1]:
# 이거하나만...
!pip install --upgrade datasets torch transformers 'accelerate>=0.26' bitsandbytes peft -qqq

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.1.0+cu118 requires torch==2.1.0, but you have torch 2.7.0 which is incompatible.
torchvision 0.16.0+cu118 requires torch==2.1.0, but you have torch 2.7.0 which is incompatible.


In [2]:
import torch
torch.__version__

'2.7.0+cu126'

In [3]:
from datasets import load_dataset

data_id = 'junelee/sharegpt_deepl_ko'
data_file_id = 'ko_dataset_2.json'
train_data = load_dataset(data_id, 
                          data_files = data_file_id, split = 'train[:1000]')

README.md:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


ko_dataset_2.json:   0%|          | 0.00/645M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [4]:
train_data.features.keys()

dict_keys(['id', 'conversations'])

In [5]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [6]:
# 토크나이져 polyglot-ko-1.3b
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("EleutherAI/polyglot-ko-1.3b")
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/164 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

In [7]:
# 데이터 전처리  from:human  from:gpt
#HUMAN:~~~~
#SYSTEM:~~~~
def preprocess(examples):
    text = ''
    for item in examples['conversations']:
        if item['from'] == 'human':
            text += f"HUMAN:{item['value']}"
        else:
            text += f"SYSTEM:{item['value']}"
    return tokenizer(text, padding='max_length', truncation=True, max_length=512)        
remove_columns = train_data.features.keys()
tokenized_datasets = train_data.map(preprocess,remove_columns = remove_columns)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [8]:
# 일반 fine_tuning
model = AutoModelForCausalLM.from_pretrained("EleutherAI/polyglot-ko-1.3b")

# LoRA 설정
from peft import LoraConfig, TaskType
peft_config = LoraConfig(
    task_type = TaskType.CAUSAL_LM, # 모델이 수행할 태스트의 종류
    inference_mode = False, # 추론 모드
    r = 8,
    lora_alpha=32,
    lora_dropout = 0.1
)
# LoRA 튜닝
from peft import get_peft_model
loRA_model = get_peft_model(model,peft_config)
# model.print_trainable_parameters()  # 적용후 조정된 파라메터

# QLoRA 양자화모델 생성
model = AutoModelForCausalLM.from_pretrained(
    'EleutherAI/polyglot-ko-1.3b',
    load_in_4bit=True,  # 4bit 양자화로 모델 가중치-->메모리 사용량 감소
    bnb_4bit_use_double_quant=True, # 중첩양자화->정밀도를 유지하면서 메모리 사용량 감소
    device_map='auto' # 여러 gpu에 모델을 자동으로 분산
)
peft_config = LoraConfig(
    task_type = TaskType.CAUSAL_LM, # 모델이 수행할 태스트의 종류
    inference_mode = False, # 추론 모드
    r = 8,
    lora_alpha=32,
    lora_dropout = 0.1
)
bnb_model = get_peft_model(model,peft_config)


config.json:   0%|          | 0.00/640 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/usr/local/lib/python3.10/dist-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


model.safetensors.index.json:   0%|          | 0.00/31.6k [00:00<?, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


...

model-00002-of-00003.safetensors:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [9]:
# bnb_model이 PEFT 모델인지 검토
from peft import PeftModel
print(isinstance(bnb_model,PeftModel))
print(hasattr(bnb_model, 'hf_quantizer'))

True
True


학습

In [10]:
from transformers import Trainer,TrainingArguments
training_args = TrainingArguments(
    output_dir = './polyglot_model',
    overwrite_output_dir = True,
    num_train_epochs = 1,    
    per_device_train_batch_size = 2,
    save_steps = 1000,
    save_total_limit = 2,
)
# 입력 텍스트를 토크나이져한후 모델이 사용할수 있도록 동적 패딩을 적용
# maksed language modeling 을 위해서 일부 토큰을 [MASK]로 변경 (BERT같은 모델에) mlm=Ture
# mlm=False GPT 같은 생성모델
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
trainer = Trainer(
    # model = loRA_model,  # LoRA 튜닝
    model = bnb_model,  #QLoRA 양자화 로라튜닝
    args = training_args,
    data_collator = data_collator,
    train_dataset = tokenized_datasets
)
trainer.train()

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:451: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


Step,Training Loss
500,2.315500


TrainOutput(global_step=500, training_loss=2.3154541015625, metrics={'train_runtime': 106.7312, 'train_samples_per_second': 9.369, 'train_steps_per_second': 4.685, 'total_flos': 3906906095616000.0, 'train_loss': 2.3154541015625, 'epoch': 1.0})

In [11]:
input_text = '마케팅에 대해서 요약해줘'
input_ids = tokenizer.encode(input_text, return_tensors='pt')
input_ids = input_ids.to(device)

max_length = 256
bnb_model = bnb_model.to(device)

test_output = bnb_model.generate(
    input_ids,
    do_sample=True,
    max_length=max_length,
    temperature = 0.7
)
print(tokenizer.decode(test_output.squeeze(),skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


마케팅에 대해서 요약해줘. 1. 광고효과 측정을 위한 여러 가지 방법 중에서 어떤 방법을 사용할 것인가? - 광고효과 측정을 위한 여러 가지 방법 중에서 어떤 방법을 사용할 것인가? - 광고효과 측정을 위한 여러 가지 방법 중에서 어떤 방법을 사용할 것인가? 1.1. 광고효과에 대한 여러 가지 측정 방법들이 있는데, 그것들 중에서 어떤 것을 사용할 것인가? - 광고효과에 대한 여러 가지 측정 방법들이 있는데, 그것들 중에서 어떤 것을 사용할 것인가? - 광고효과에 대한 여러 가지 측정 방법들이 있는데, 그것들 중에서 어떤 것을 사용할 것인가? 1.2. 어떤 측정 방법을 사용할 것인가? - 어떤 측정 방법을 사용할 것인가? - 광고효과에 대한 어떤 측정 방법을 사용할 것인가? - 어떤 측정 방법을 사용할 것인가? - 어떤 측정 방법을 사용할 것인가? 1.3. 어떤 측정 방법을 사용할 것인가? - 어떤 측정 방법을 사용할 것인가? - 어떤 측정 방법을 사용할 것인가? - 어떤 측정 방법을 사용할 것인가? - 어떤 측정 방법을 사용할 것인가? 1.4


In [27]:
print(tokenizer.decode(test_output.squeeze(),skip_special_tokens=True) )

대한민국에 대해서 요약해줘'라는 말에 저는 마음을 졸이며 대답했습니다. '내가 여기서 뭘 하고 있는거야? 나는 지금까지 모든 것을 다해봤어. 그리고 아직 할 일이 많이 남아있어.' "네가 여기서 뭐하는 거냐고 묻는다면, 나는 항상 네가 여기서 뭔가를 하고 있을 것이라고 생각했기 때문이다."라고 그녀는 대답했습니다. "저는 항상 여기 있다고 생각했어요. 그리고 항상 여기 있었어요. 그리고 여기 있을 거라고 생각해요. 하지만 저는 항상 여기 있었어요."라고 그녀가 대답했습니다. "하지만 당신이 여기 있는 이유는 무엇인가요? 여기 있는 유일한 이유는 무엇인가요?"라고 그녀가 물었습니다. "저는 항상 여기 있었어요. 그리고 항상 여기 있을 거라고 생각했어요. 하지만 여기 있는 유일한 이유는 내가 여기 있는 유일한 이유이기 때문이에요."라고 그녀는 대답했습니다. "그 유일한 이유는 무엇인가요?"라고 그녀가 물었습니다. "저는 항상 여기 있을 거라고 생각했어요. 그리고 항상 여기 있을 거라고 생각해요.


In [16]:
# # 현재 환경 저장
# !pip freeze > requirements.txt

In [2]:
# # 환경 로드
# !pip install -r requirements.txt

  Using cached accelerate-1.7.0-py3-none-any.whl.metadata (19 kB)
  Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl.metadata (5.9 kB)
  Using cached aiohttp-3.11.18-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.7 kB)
  Using cached aiosignal-1.3.2-py2.py3-none-any.whl.metadata (3.8 kB)
  Using cached async_timeout-5.0.1-py3-none-any.whl.metadata (5.1 kB)
  Using cached datasets-3.6.0-py3-none-any.whl.metadata (19 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached frozenlist-1.6.0-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (16 kB)
  Using cached fsspec-2025.3.0-py3-none-any.whl.metadata (11 kB)
  Using cached huggingface_hub-0.31.4-py3-none-any.whl.metadata (13 kB)
  Using cached multidict-6.4.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.3 kB)
  Using cached multiprocess-0.70.16-py310-none-any.whl.metadata (7.2 kB)
  Using cached nvidia_cu

양자화

In [ ]:
#bitsandbytes 라이브러 

In [31]:
f'{model.get_memory_footprint() / 1_000_000_000}GB'

'5.327241344GB'

In [8]:
# 4bit 중첩양자화 - 성능이 우수
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained(
    'EleutherAI/polyglot-ko-1.3b',
    load_in_4bit=True,  # 4bit 양자화로 모델 가중치-->메모리 사용량 감소
    bnb_4bit_use_double_quant=True, # 중첩양자화->정밀도를 유지하면서 메모리 사용량 감소
    device_map='auto' # 여러 gpu에 모델을 자동으로 분산
)
f'{model.get_memory_footprint() / 1_000_000_000}GB'

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

'0.851681408GB'

In [2]:
0.851681408/5.327241344

0.15987287847569312